In [ ]:
import cv2
import numpy as np
from PIL import Image
import os

# Load image
image_list = [
    r"d:\3dModel\Hueforge_image\kyockcho\PROCESSED\50-67614783_p2.png",
]
os.makedirs(r"r:\out", exist_ok=True)
for input_path in image_list:
    output_path = f"r:/out/{os.path.splitext(os.path.basename(input_path))[0]}.png"
    # output_path = 
    gap = 101  # Gap in pixels

    # Load original image
    img = cv2.imread(input_path, cv2.IMREAD_UNCHANGED)
    if img.shape[2] == 4:  # Check if the image has an alpha channel
        # Split the RGBA image into its channels
        b, g, r, a = cv2.split(img)
        # Merge the RGB channels
        rgb = cv2.merge([b, g, r])
        # Normalize the alpha channel to range [0, 1]
        alpha = a / 255.0
        # Create a white base image of the same size as the input image
        white_base = np.ones_like(rgb, dtype=np.uint8) * 255
        # Overlay the RGB image on the white base using the alpha channel
        img = (rgb * alpha[:, :, None] + white_base * (1 - alpha[:, :, None])).astype(np.uint8)
    h, w = img.shape[:2]

    # Blur the original image to help the mask cover anti-aliased edges
    blurred_img = cv2.GaussianBlur(img, (11, 11), 0)

    # Flood fill the blurred image to detect background only
    mask = np.zeros((h + 2, w + 2), np.uint8)
    flooded = blurred_img.copy()
    diff  = (1, 1, 1)  # Difference for flood fill
    cv2.floodFill(flooded, mask, (0, 0), (0, 0, 0), loDiff=diff, upDiff=diff)
    cv2.floodFill(flooded, mask, (w - 1, h - 1), (0, 0, 0), loDiff=diff, upDiff=diff)
    cv2.floodFill(flooded, mask, (w - 1, 0), (0, 0, 0), loDiff=diff, upDiff=diff)
    # cv2.floodFill(flooded, mask, (w // 2, h - 1), (0, 0, 0), loDiff=diff, upDiff=diff)
    cv2.floodFill(flooded, mask, (0, h - 1), (0, 0, 0), loDiff=diff, upDiff=diff)


    # Create foreground mask from difference between original and flood filled (background)
    diff = cv2.absdiff(cv2.cvtColor(blurred_img, cv2.COLOR_BGR2GRAY), 
                    cv2.cvtColor(flooded, cv2.COLOR_BGR2GRAY))
    _, object_mask = cv2.threshold(diff, 1, 255, cv2.THRESH_BINARY)

    # Blur the object_mask
    blurred_mask = cv2.GaussianBlur(object_mask, (gap, gap), 0)
    # blurred_mask = cv2.blur(object_mask.astype(np.float32), (gap, gap), 0)

    # Threshold the blurred mask
    _, thresholded_mask = cv2.threshold(blurred_mask, 254, 255, cv2.THRESH_BINARY)

    # Invert mask to get alpha channel: object opaque, outside transparent
    alpha = 255 - thresholded_mask.astype(np.uint8)

    # Merge into RGBA
    b, g, r = cv2.split(img)
    rgba = cv2.merge([r, g, b, alpha])

    # Save result
    out = Image.fromarray(rgba)
    out.save(output_path, "PNG")
    print(f"Saved {output_path}")
out